# **PolitiFact WebScrapping Module**



**ScrapeFunction**

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

authors = []
statements = []
targets = []
text_1 = []
url_1 = []

def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page={}&'.format(page_num)
    webpage = requests.get(URL) 
    #time.sleep(3)
    soup = BeautifulSoup(webpage.text, "html.parser") 
    statement_footer =  soup.find_all('footer',attrs={'class':'m-statement__footer'}) 
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'})
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})
    target = soup.find_all('div', attrs={'class':'m-statement__meter'}) 
    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
        authors.append(full_name)
    for i in statement_quote:
      for link in i.find_all('a',href=True):
        title = i.text.strip()
        statements.append(title)
        url_1.append("https://www.politifact.com"+link['href'])
    for link1 in url_1:
      webpage1 = requests.get(link1)
      soup1 = BeautifulSoup(webpage1.text, "html.parser") 
      statement_text = soup1.find_all('article',attrs={'m-textblock'})
      for k in statement_text:
        lines = k.text.strip()
        #lines = str(lines)
        text1= lines.split('\n')
        text2=text1[0:5]
        text_1.append(text2)  
    for i in target:
        fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(fact)



# Target Column Convert To 0 And 1 Function

In [2]:
def getBinaryNumTarget(text):
    if text == 'true':
        return 1
    elif text == 'mostly-true':
        return 1
    else:
        return 0

# Enter Start Page No And End Page No

In [5]:
df = pd.DataFrame()
startpage = int(input("enter start page no:"))
endpage = int(input("enter end page no:"))
for i in range(startpage,endpage+1):
  authors.clear()
  statements.clear()
  targets.clear()
  text_1.clear()
  url_1.clear()
  scrape_website(i)
  data = pd.DataFrame()
  data['Title'] = statements
  data['Author'] = authors
  data['Text'] = text_1
  data['Target'] = targets
  df=df.append(data,ignore_index=True)
df['Label']=df['Target'].apply(getBinaryNumTarget)
filename='pageno{}to{}.csv'.format(startpage,endpage)
df.to_csv(filename)  

enter start page no:21
enter end page no:30
